In [17]:
import spiceypy as spice

In [18]:
# Load SPICE kernels
spice.furnsh('./kernels/lsk/naif0012.tls')
spice.furnsh('./kernels/spk/vgr1_jup230.bsp')
spice.furnsh('./kernels/spk/vgr1_sat337.bsp')
spice.furnsh('./kernels/spk/vgr1.x2100.bsp')
